In [1]:
import os
import time
import pyodbc
import textwrap
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from IPython.core.display import HTML

from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

# Warning control
import warnings
warnings.filterwarnings("ignore")

- Install relavant pkgs
- ! pip install --upgrade langchain langchain_community langchain-openai
- ! pip install neo4j

In [2]:
load_dotenv('.env.studiomac', override=True)

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

AZURE_RESOURCE_NAME=os.getenv('AZURE_RESOURCE_NAME')
AZURE_ENDPOINT=os.getenv('AZURE_ENDPOINT')
AZURE_API_KEY1=os.getenv('AZURE_API_KEY1')
AZURE_API_KEY2=os.getenv('AZURE_API_KEY2')
AZURE_EMBED_DEPLOYMENT=os.getenv('AZURE_EMBED_DEPLOYMENT')
AZURE_CHATGPT_DEPLOYMENT=os.getenv('AZURE_CHATGPT_DEPLOYMENT')
AZURE_OPENAI_API_VERSION=os.getenv('AZURE_OPENAI_API_VERSION')

SR_DATABASE=os.getenv('SR_DATABASE')
SR_SERVER=os.getenv('SR_SERVER')
SR_USERNAME=os.getenv('SR_USERNAME')
SR_PASSWORD=os.getenv('SR_PASSWORD')
SR_DRIVER=os.getenv('SR_DRIVER')

## Acquire the Data from MS SQL

In [3]:
# azquire service request data from MSSQL
connect_str="Driver={%s};Server=%s;Database=%s;uid=%s;pwd=%s" % (SR_DRIVER,
                                                                 SR_SERVER,
                                                                 SR_DATABASE,
                                                                 SR_USERNAME,
                                                                 SR_PASSWORD)
# in this case, turn off the word-length filter
def select_tbl_customized(where_condition=None):
    strSQL="""WITH CTE0 AS (
                SELECT * FROM [Weintek_Web].[dbo].[SR_Files]
                WHERE SR_Message_Uid = 1
                ), CTE1 AS (
                SELECT 
                it.SR_Uid, it.SR_Message_Uid, it.SR_Uid+'_'+ CONVERT(VARCHAR,it.SR_Message_Uid) AS PK,it.User_Uid, h.Created_UserIP, it.Post_Date, it.SR_Message, it.SR_Reply, it.SR_Replier, it.SR_ReplyDate, it.SR_ReplyFile, it.SR_ReplyFileName,
                h.SR_Subject, h.SR_Status, h.EmailNotify, h.SR_Category, h.Created_Date, h.Posts, h.Replies, h.SN, h.MODEL, h.OS, h.Firmware_Version, h.PLC, h.Driver, h.PLC_IF, 
                h.HMI_COMport, h.Baud_Rate, h.Data_bits, h.Parity, h.Stop_bits, h.Delay, h.LastUpdate_User, h.LastUpdate_Date, h.SR_PType, h.SR_PUid, h.SR_Replier as SR_ORG_Replier,
                f.SR_Files1, f.SR_Files1_Name, f.SR_Files1_Date,
                CASE WHEN h.LastUpdate_Date >= DATEADD(DAY,-14,GETDATE()) and h.SR_Status = 'Replied' THEN 'N' ELSE 'Y' END AS Recall_Replied
                FROM [Weintek_Web].[dbo].[SR_Message] it
                LEFT JOIN [Weintek_Web].[dbo].[SR_Thread] h ON it.SR_Uid = h.SR_Uid
                LEFT JOIN [Weintek_Web].[dbo].[SR_Files] f ON it.SR_Uid = f.SR_Uid AND it.SR_Message_Uid = f.SR_Message_Uid
                WHERE h.SR_Status in ('Closed','Replied') AND h.SR_Replier <> 'seanhuang'
                AND it.SR_Uid not in (SELECT CTE0.SR_Uid FROM CTE0)
                /*AND LEN(it.SR_Message) > 60 AND LEN(it.SR_Reply) > 60*/
                )
                SELECT * FROM CTE1
                WHERE CTE1.Recall_Replied = 'Y'"""
    if where_condition is not None:
        strSQL+=f" AND {where_condition}"
    conn = pyodbc.connect(connect_str)
    cursor = conn.cursor()
    df = pd.read_sql(strSQL,conn)
    cursor.close()
    conn.close()
    return df

In [4]:
df_sr = select_tbl_customized()
df_sr.head(5)

,SR_Uid,SR_Message_Uid,PK,User_Uid,Created_UserIP,Post_Date,SR_Message,SR_Reply,SR_Replier,SR_ReplyDate,...,Delay,LastUpdate_User,LastUpdate_Date,SR_PType,SR_PUid,SR_ORG_Replier,SR_Files1,SR_Files1_Name,SR_Files1_Date,Recall_Replied
0,20091207_0004,1,20091207_0004_1,Hades,77.78.151.5,2009-12-08 08:58:44.913,"<br />\r\nDear sirs,<br />\r\n<br />\r\nWe hav...","Dear Sir, <br />\r\n1. MT6070iH is not able to...",Cino,2009-12-15 10:23:20.980,...,,Cino,2009-12-15 10:23:20.980,MT6000/8000,MT6070iH,Tina0604,None,None,NaT,Y
1,20091207_0004,2,20091207_0004_2,Hades,77.78.151.5,2009-12-08 20:09:46.443,"&nbsp;Dear Tina,<br />\n&nbsp;Thank you for yo...","Dear Stefan,<br />\r\n<br />\r\nFor MT6070iH t...",Cino,2009-12-15 10:11:30.183,...,,Cino,2009-12-15 10:23:20.980,MT6000/8000,MT6070iH,Tina0604,None,None,NaT,Y
2,20091208_0001,1,20091208_0001_1,Budi,192.168.1.254,2009-12-08 11:19:48.380,"Dear Sir,<br />\r\n<br />\r\nHow are you?<br /...",MT8000 series 123gjg,Cino,2010-01-25 11:18:27.970,...,,Cino,2010-01-25 11:18:53.563,MT6000/8000,MT6050i,nicolas,None,None,NaT,Y
3,20091208_0001,2,20091208_0001_2,Budi,192.168.1.254,2009-12-08 14:28:47.287,"Dear Sir,<br />\r\n<br />\r\nvery thank you fo...",123 re tyytjhgj,Cino,2010-01-25 11:18:53.563,...,,Cino,2010-01-25 11:18:53.563,MT6000/8000,MT6050i,nicolas,None,None,NaT,Y
4,20091208_0001,3,20091208_0001_3,Budi,192.168.1.254,2009-12-08 14:29:05.897,"Dear Sir,<br />\r\n<br />\r\nvery thank you fo...",123 re tyytjhgj,Cino,2010-01-25 11:18:49.767,...,,Cino,2010-01-25 11:18:53.563,MT6000/8000,MT6050i,nicolas,None,None,NaT,Y


- acquire thread id list and print the first thread id

In [5]:
thread_id_list = set(df_sr.SR_Uid.to_list())
thread_id_list = sorted(list(thread_id_list))
first_thread_id = thread_id_list[0]
print(first_thread_id)

20091207_0004


## Establish nodes and properties

In [6]:
HTML('<img src=".\imgs\kg1.png" alt="kg Image" width="500"/>')

- get first thread data

In [7]:
df_first = df_sr[df_sr.SR_Uid==first_thread_id].copy()
df_first = df_first.sort_values(by='SR_Message_Uid')
df_first.head()

,SR_Uid,SR_Message_Uid,PK,User_Uid,Created_UserIP,Post_Date,SR_Message,SR_Reply,SR_Replier,SR_ReplyDate,...,Delay,LastUpdate_User,LastUpdate_Date,SR_PType,SR_PUid,SR_ORG_Replier,SR_Files1,SR_Files1_Name,SR_Files1_Date,Recall_Replied
0,20091207_0004,1,20091207_0004_1,Hades,77.78.151.5,2009-12-08 08:58:44.913,"<br />\r\nDear sirs,<br />\r\n<br />\r\nWe hav...","Dear Sir, <br />\r\n1. MT6070iH is not able to...",Cino,2009-12-15 10:23:20.980,...,,Cino,2009-12-15 10:23:20.980,MT6000/8000,MT6070iH,Tina0604,None,None,NaT,Y
1,20091207_0004,2,20091207_0004_2,Hades,77.78.151.5,2009-12-08 20:09:46.443,"&nbsp;Dear Tina,<br />\n&nbsp;Thank you for yo...","Dear Stefan,<br />\r\n<br />\r\nFor MT6070iH t...",Cino,2009-12-15 10:11:30.183,...,,Cino,2009-12-15 10:23:20.980,MT6000/8000,MT6070iH,Tina0604,None,None,NaT,Y


- Define a function to prepare metadata

In [8]:
def build_sr_metadata(df:pd.DataFrame):
    thread_metadata=[] # thread node
    userProfile_metada=[] # user node
    userModel_metadata=[] # model node
    messages_metadata=[] # message node
    thread_id_list = set(df.SR_Uid.to_list())
    thread_id_list = sorted(list(thread_id_list))
    # scan each thread id
    for idx1, sr_uid in enumerate(thread_id_list):
        df_sr_uid = df[df.SR_Uid == sr_uid].sort_values(by='SR_Message_Uid')
        first_row = df_sr_uid.iloc[0]
        thread_metadata.append({"ThreadId":sr_uid,
                                "Subject":first_row['SR_Subject'],
                                "CreateDate":first_row["Created_Date"].isoformat(),
                                "Category":first_row["SR_Category"]})
        user_uid = first_row['User_Uid']
        userProfile_metada.append({"UserId":sr_uid + "-" + user_uid,
                                   "ThreadId":sr_uid,
                                   "username":user_uid,
                                   "country":'',
                                   "userIP":first_row["Created_UserIP"]})
        hmi_model = first_row['SR_PUid']
        plc_model = first_row['PLC']
        userModel_metadata.append({"ModelId":sr_uid + "-" + hmi_model + "-" + plc_model,
                                   "ThreadId":sr_uid,
                                   "HMI_Model":hmi_model,
                                   "HMI_COMport":first_row['HMI_COMport'],
                                   "Firmware_Version":first_row['Firmware_Version'],
                                   "OS_Version":first_row['OS'],
                                   "SerialNo":first_row['SN'],
                                   "PLC_Model":plc_model,
                                   "Driver":first_row['Driver'],
                                   "PLC_Connection":first_row['PLC_IF']})
        # scan each message in this thread, here we add source which is threadId. source will be used in RetrieveQA module
        messages = []
        for idx2, row in df_sr_uid.iterrows():
            messages.append({"MessageOrder":row['SR_Message_Uid'],
                            "source":sr_uid,
                            "PK":row['PK'],
                            "PostDate":row['Post_Date'].isoformat(),
                            "MessageText":row['SR_Message'],
                            "ReplyText":row['SR_Reply'],
                            "Replier":row['SR_Replier'],
                            "ReplyDate":row['SR_ReplyDate'].isoformat(),
                            "ReplyFile":row['SR_ReplyFile']})
        messages_metadata.append(messages)
    return thread_metadata,userProfile_metada,userModel_metadata,messages_metadata

- test this function on df_first

In [9]:
thread_metadata,userProfile_metada,userModel_metadata,messages_metadata = build_sr_metadata(df_first)
print(thread_metadata)
print(userProfile_metada)
print(userModel_metadata)
print(messages_metadata)

[{'ThreadId': '20091207_0004', 'Subject': 'Pass-through mode', 'CreateDate': '2009-12-07T23:00:06.053000', 'Category': 'Software'}]
[{'UserId': '20091207_0004-Hades', 'ThreadId': '20091207_0004', 'username': 'Hades', 'country': '', 'userIP': '77.78.151.5'}]
[{'ModelId': '20091207_0004-MT6070iH-Mitsubishi FX1N', 'ThreadId': '20091207_0004', 'HMI_Model': 'MT6070iH', 'HMI_COMport': '   ', 'Firmware_Version': 'MT8xxx(S3C) build 20091002', 'OS_Version': '', 'SerialNo': '09110371', 'PLC_Model': 'Mitsubishi FX1N', 'Driver': '', 'PLC_Connection': 'RS485 4W'}]
[[{'MessageOrder': 1, 'source': '20091207_0004', 'PK': '20091207_0004_1', 'PostDate': '2009-12-08T08:58:44.913000', 'MessageText': '<br />\r\nDear sirs,<br />\r\n<br />\r\nWe have used MT506TV for 3 years.<br />\r\nNow I&nbsp;am exploring your new model MT6070iH, and I am trying to realize a connection between PC, HMI and PLC. HMI-PLC communication seems \r\r\nOK, but the only posible connection for HMI project&nbsp;downloading I’m realiz

# Create Graph Nodes by using metadata

- connect to Neo4 db

In [10]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE, timeout=86400
)

- create thread node

In [11]:
merge_thread_node_query = """
MERGE(mergedChunk:Thread {ThreadId: $threadParam.ThreadId})
    ON CREATE SET 
        mergedChunk.Subject = $threadParam.Subject,
        mergedChunk.CreateDate = $threadParam.CreateDate, 
        mergedChunk.Category = $threadParam.Category
RETURN mergedChunk
"""

kg.query(merge_thread_node_query, 
         params={'threadParam':thread_metadata[0]})

[{'mergedChunk': {'Category': 'Software',
   'ThreadId': '20091207_0004',
   'CreateDate': '2009-12-07T23:00:06.053000',
   'Subject': 'Pass-through mode'}}]

In [12]:
# search query
cypher = """MATCH (n:Thread) 
            RETURN n.ThreadId as ThreadId"""
kg.query(cypher)

[{'ThreadId': '20091207_0004'}]

- create user profile node

In [13]:
merge_user_node_query = """
MERGE(mergedChunk:User {UserId: $userParam.UserId})
    ON CREATE SET 
        mergedChunk.username = $userParam.username,
        mergedChunk.country = $userParam.country, 
        mergedChunk.userIP = $userParam.userIP,
        mergedChunk.ThreadId = $userParam.ThreadId
RETURN mergedChunk
"""

kg.query(merge_user_node_query, 
         params={'userParam':userProfile_metada[0]})


[{'mergedChunk': {'country': '',
   'ThreadId': '20091207_0004',
   'UserId': '20091207_0004-Hades',
   'userIP': '77.78.151.5',
   'username': 'Hades'}}]

- create model node

In [14]:
merge_model_node_query = """
MERGE(mergedChunk:Model {ModelId: $modelParam.ModelId})
    ON CREATE SET 
        mergedChunk.HMI_Model = $modelParam.HMI_Model,
        mergedChunk.HMI_COMport = $modelParam.HMI_COMport, 
        mergedChunk.Firmware_Version = $modelParam.Firmware_Version, 
        mergedChunk.OS_Version = $modelParam.OS_Version, 
        mergedChunk.SerialNo = $modelParam.SerialNo, 
        mergedChunk.PLC_Model = $modelParam.PLC_Model, 
        mergedChunk.Driver = $modelParam.Driver, 
        mergedChunk.PLC_Connection = $modelParam.PLC_Connection,
        mergedChunk.ThreadId = $modelParam.ThreadId
RETURN mergedChunk
"""

kg.query(merge_model_node_query, 
         params={'modelParam':userModel_metadata[0]})

[{'mergedChunk': {'HMI_Model': 'MT6070iH',
   'SerialNo': '09110371',
   'PLC_Connection': 'RS485 4W',
   'ThreadId': '20091207_0004',
   'HMI_COMport': '   ',
   'Driver': '',
   'OS_Version': '',
   'PLC_Model': 'Mitsubishi FX1N',
   'ModelId': '20091207_0004-MT6070iH-Mitsubishi FX1N',
   'Firmware_Version': 'MT8xxx(S3C) build 20091002'}}]

- create message node

In [15]:
merge_message_node_query = """
MERGE(mergedChunk:Message {MsgId: $msgParam.PK})
    ON CREATE SET 
        mergedChunk.MessageOrder = $msgParam.MessageOrder,
        mergedChunk.source = $msgParam.source, 
        mergedChunk.PostDate = $msgParam.PostDate, 
        mergedChunk.MessageText = $msgParam.MessageText, 
        mergedChunk.ReplyText = $msgParam.ReplyText, 
        mergedChunk.Replier = $msgParam.Replier, 
        mergedChunk.ReplyDate = $msgParam.ReplyDate, 
        mergedChunk.ReplyFile = $msgParam.ReplyFile
RETURN mergedChunk
"""

# scan each message
for metadata in messages_metadata[0]:
    kg.query(merge_message_node_query, 
            params={'msgParam':metadata})

- Create a uniqueness constraint to avoid duplicate thread

In [16]:
# for thread
kg.query("""
CREATE CONSTRAINT unique_thread IF NOT EXISTS 
    FOR (t:Thread) REQUIRE t.ThreadId IS UNIQUE
""")
# for user
kg.query("""
CREATE CONSTRAINT unique_user IF NOT EXISTS 
    FOR (u:User) REQUIRE u.UserId IS UNIQUE
""")
# for model
kg.query("""
CREATE CONSTRAINT unique_model IF NOT EXISTS 
    FOR (m:Model) REQUIRE m.ModelId IS UNIQUE
""")
# for message
kg.query("""
CREATE CONSTRAINT unique_message IF NOT EXISTS 
    FOR (m:Message) REQUIRE m.MsgId IS UNIQUE
""")


[]

- loop all threads to create all nodes

In [17]:
thread_metadata,userProfile_metada,userModel_metadata,messages_metadata = build_sr_metadata(df_sr)

In [18]:
# to see the number of unique items
print(len(thread_metadata))
print(len(userProfile_metada))
print(len(userModel_metadata))
print(len(messages_metadata))
print("====================")
num_thread_node = len(set([node['ThreadId'] for node in thread_metadata]))
num_user_node = len(set([node['UserId'] for node in userProfile_metada]))
num_model_node = len(set([node['ModelId'] for node in userModel_metadata]))
num_message_node = sum([len(node_grp) for node_grp in messages_metadata])
print(num_thread_node)
print(num_user_node)
print(num_model_node)
print(num_message_node)
print(f"total nodes: {num_thread_node+num_user_node+num_model_node+num_message_node}")

9963
9963
9963
9963
9963
9963
9963
20975
total nodes: 50864


In [19]:
# scan data, totally about 37k nodes (172 min)
for idx, thread_data in enumerate(tqdm(thread_metadata,desc="Processing threadId",unit="threads")):
     # if idx > 200:
     #      break
     kg.query(merge_thread_node_query, 
          params={'threadParam':thread_data})
     # scan each user
     user_data = userProfile_metada[idx]
     kg.query(merge_user_node_query, 
          params={'userParam':user_data})
     # scan each model
     model_data = userModel_metadata[idx]
     kg.query(merge_model_node_query, 
          params={'modelParam':model_data})
     # scan each message
     messages_data = messages_metadata[idx]
     for metadata in messages_data:
          kg.query(merge_message_node_query, 
                    params={'msgParam':metadata})

Processing threadId: 100%|██████████| 9963/9963 [3:12:46<00:00,  1.16s/threads]  


In [20]:
# query a given thread
kg.query("""MATCH(t:Thread), (u:User), (mod:Model), (msg:Message)
            WHERE t.ThreadId = u.ThreadId and t.ThreadId = mod.ThreadId and t.ThreadId = msg.source
            WITH t, u, mod, msg
            WHERE t.ThreadId = '20100423_0001'
            RETURN t, mod, collect(msg) as qa_paris
            ORDER BY t.ThreadId DESC""")

[{'t': {'Category': 'Software',
   'ThreadId': '20100423_0001',
   'CreateDate': '2010-04-23T03:04:00.913000',
   'Subject': 'Can not open mtp file'},
  'mod': {'HMI_Model': 'MT6070iH',
   'SerialNo': '09093683',
   'PLC_Connection': '  ',
   'ThreadId': '20100423_0001',
   'HMI_COMport': '   ',
   'Driver': 'SIEMENS_S7_200.so',
   'OS_Version': '',
   'PLC_Model': 'S7-200',
   'ModelId': '20100423_0001-MT6070iH-S7-200',
   'Firmware_Version': '?'},
  'qa_paris': [{'Replier': 'Tina0604',
    'MessageOrder': 1,
    'ReplyDate': '2010-04-23T11:50:04.720000',
    'source': '20100423_0001',
    'MessageText': 'I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />',
    'PostDate': '2010-04-23T03:04:00.930000',
    'MsgId': '20100423_0001_1',
    'ReplyText': '<p

## Create a vector index

In [21]:
kg.query("""
         CREATE VECTOR INDEX `form_user_message` IF NOT EXISTS
          FOR (m:Message) ON (m.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`:3072,
            `vector.similarity_function`:'cosine'    
         }}
""")

[]

In [22]:
kg.query("SHOW INDEXES WHERE name='form_user_message'")

[{'id': 10,
  'name': 'form_user_message',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Message'],
  'properties': ['textEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

### Calculate the vector embedding for message text with Azure OpenAI

- batch running for textEmbedding to avoid timeout and connection error 

In [23]:
batch_size = 1000 # about 3 min/batch
max_retry = 3
try_cnt = 0
batch_cnt = 0
while try_cnt < max_retry:
    print("Try Batch %d..." % (batch_cnt+1))
    try:
        result = kg.query("""
                          MATCH (msg:Message) WHERE msg.textEmbedding IS NULL
                          WITH msg LIMIT %d
                          WITH msg, genai.vector.encode(
                            msg.MessageText, 
                            "AzureOpenAI", 
                            {
                              token: $azureOpenAiApiKey, 
                              endpoint: $azureOpenAiEndpoint,
                              resource: $azureOpenAiResource,
                              deployment: $azureDeployment
                            }) AS vector
                          CALL db.create.setNodeVectorProperty(msg, "textEmbedding", vector)
                          RETURN msg
                          """ % (batch_size), 
                          params={"azureOpenAiApiKey":AZURE_API_KEY1, 
                                  "azureOpenAiEndpoint": AZURE_ENDPOINT, 
                                  "azureOpenAiResource":AZURE_RESOURCE_NAME, 
                                  "azureDeployment":AZURE_EMBED_DEPLOYMENT})
        batch_cnt += 1
        if len(result) < batch_size:
            print("Completed.")
            break
    except Exception as e:
        try_cnt += 1
        print(f"Try {try_cnt} with error: {e}")
        print("Waiting....1 min")
        time.sleep(60)
else:
    print("Failed after max retries.")


Try Batch 1...
Try Batch 2...
Try Batch 3...
Try Batch 4...
Try Batch 5...
Try Batch 6...
Try Batch 7...
Try Batch 8...
Try Batch 9...
Try Batch 10...
Try Batch 11...
Try Batch 12...
Try Batch 13...
Try Batch 14...
Try Batch 15...
Try Batch 16...
Try Batch 17...
Try Batch 18...
Try Batch 19...
Try Batch 20...
Try Batch 21...
Completed.


In [24]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Thread {ThreadId: STRING, Subject: STRING, CreateDate: STRING, Category: STRING}
User {ThreadId: STRING, UserId: STRING, username: STRING, country: STRING, userIP: STRING}
Model {ThreadId: STRING, ModelId: STRING, HMI_Model: STRING, HMI_COMport: STRING, Firmware_Version: STRING, OS_Version: STRING, SerialNo: STRING, PLC_Model: STRING, Driver: STRING, PLC_Connection: STRING}
Message {MsgId: STRING, MessageOrder: INTEGER, source: STRING, PostDate: STRING, MessageText: STRING, ReplyText: STRING, Replier: STRING, ReplyDate: STRING, textEmbedding: LIST, ReplyFile: STRING}
Relationship properties:

The relationships:



### Use Similarity to find relevant messages

In [25]:
def neo4j_vector_search(question, VECTOR_INDEX_NAME = "form_user_message"):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    WITH genai.vector.encode(
      $question, 
      "AzureOpenAI", 
      {
        token: $azureOpenAiApiKey, 
        endpoint: $azureOpenAiEndpoint,
        resource: $azureOpenAiResource,
        deployment: $azureDeployment
      }) AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
    RETURN score, node.MessageText AS Question, node.ReplyText As Answer
  """
  similar = kg.query(vector_search_query, 
                     params={
                      'question': question, 
                      "azureOpenAiApiKey":AZURE_API_KEY1, 
                      "azureOpenAiEndpoint": AZURE_ENDPOINT, 
                      "azureOpenAiResource":AZURE_RESOURCE_NAME, 
                      "azureDeployment":AZURE_EMBED_DEPLOYMENT,
                      'index_name':VECTOR_INDEX_NAME, 
                      'top_k': 5})
  return similar

In [26]:
search_results = neo4j_vector_search(
    'I can not open my file with your easybuilder software.'
)
print(search_results)

[{'score': 0.8800181746482849, 'Question': 'I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />', 'Answer': '<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />\r\n&nbsp;</p>'}, {'score': 0.8279533386230469, 'Question': '<p>Okay thanks,</p><br><p>Can you also look to the easybuilder file. I cannot open this file anymore.<br></p>', 'Answer': 'Dear Sir,<br />\r\n<br />\r\nWhat version did you use to make this project?<br />\r\nBefore it can’t open with V6.02.01, what did you do?<br />\r\n<br />\r\nBest Regards,<br />\r\nFinn<br />\r\n<br type="_moz" />'}, {'score': 0.819596529006958, 'Question': 'Hello! I have bought toch panel MT8071ip. I downloaded the project from the panel that was record

## Set up the Langchain RAG workflow to chat with the data

- Use Azure OpenAIEmbeddings

In [27]:
Embeddings = AzureOpenAIEmbeddings(
    azure_deployment=AZURE_EMBED_DEPLOYMENT,
    openai_api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_API_KEY1,
)

VECTOR_INDEX_NAME = "form_user_message"
VECTOR_NODE_LABEL = "Message"
VECTOR_SOURCE_PROPERTY1 = 'MessageText'
VECTOR_SOURCE_PROPERTY2 = "RepleyText"
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=Embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY1],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

retriever = neo4j_vector_store.as_retriever(search_kwargs={"k": 15})

In [28]:
# search by retriever
result = retriever.invoke("I can not open my file with your easybuilder software.")
result

[Document(metadata={'MsgId': '20100423_0001_1', 'Replier': 'Tina0604', 'PostDate': '2010-04-23T03:04:00.930000', 'source': '20100423_0001', 'MessageOrder': 1, 'ReplyText': '<p>Which EB8000 version you used before? have you open your project with EB8000v4.00 or v4.02beta version?&nbsp;<br />\r\n&nbsp;</p>', 'ReplyDate': '2010-04-23T11:50:04.720000'}, page_content='\nMessageText: I can not open my file with your easybuilder software.&nbsp; I have tried several differnet version and i always receive and error saying &quot;Must use a newer tool to open&quot;.<br />\r\n<br />\r\nHow do i find out what version i need, unable to get ahold of oem who supplied.<br />'),
 Document(metadata={'MsgId': '20190109_0001_4', 'Replier': 'finnliu', 'PostDate': '2019-01-09T08:09:56.937000', 'source': '20190109_0001', 'MessageOrder': 4, 'ReplyText': 'Dear Sir,<br />\r\n<br />\r\nWhat version did you use to make this project?<br />\r\nBefore it can’t open with V6.02.01, what did you do?<br />\r\n<br />\r\nB

- Set up a RetrievalQAWithSourcesChain to carry out question answering
- You can check out the LangChain documentation for this chain [here](https://learn.microsoft.com/en-us/samples/azure-samples/container-apps-openai/container-apps-openai/)


In [29]:
llm = AzureChatOpenAI(
            openai_api_type="azure",
            openai_api_version=AZURE_OPENAI_API_VERSION,
            openai_api_key=AZURE_API_KEY1,
            azure_endpoint=AZURE_ENDPOINT,
            temperature=0.7,
            max_tokens=4096,
            azure_deployment=AZURE_CHATGPT_DEPLOYMENT,
            streaming=True,
            max_retries=3,
            timeout=60,
        )

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever
)

def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    # with streaming
    # chunks = []
    # for chunk in chain.stream({"question": question}):
    #     yield chunk['answer']
    #     time.sleep(0.05)
        # print(chunk['answer'], end="|", flush=True)
    ## with no streaming
    response = chain({"question": question},
                    return_only_outputs=True)
    print(textwrap.fill(response['answer'], 150))

- ask gpt a question

In [30]:
prettychain("I can not open my file with your easybuilder software. please help me")

To resolve the issue of not being able to open your file with the EasyBuilder software, it appears that the problem is related to the version of the
software you are using. Several users have reported similar issues where older files or files from different versions cannot be opened with their
current version of EasyBuilder.  Here are some steps you can try:  1. **Identify the Correct Version**: Ensure you are using the latest version of
EasyBuilder Pro. You might need a newer version than the one you currently have. For example, one user reported that even with EasyBuilder Pro
6.03.01.239, they encountered issues (
